<a href="https://colab.research.google.com/github/MarcosVeniciu/Producao-de-cafe-MG/blob/main/Randon_Florest_Coffee_Production_Prediction_with_Machine_Learning_in_Minas_Gerais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV

# 1) Habilita widgets
!pip install -q ipywidgets
from google.colab import output
output.enable_custom_widget_manager()

# 2) Modo notebook do Plotly
!pip install -q plotly
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# 3) Renderer para Colab
import plotly.io as pio
pio.renderers.default = 'colab'

# 9. Divisão treino/teste

In [ ]:
dataset = pd.read_csv('/content/dataset_v3.csv')
print(f"Total de dados: {len(dataset)}")

- Log-transform em Área colhida, Quantidade (em mil toneladas) e Valor da produção.

In [ ]:
log_cols = [
  'Área colhida (Hectares)',
  'target',
  'Valor da produção (Mil Reais)'
]

# Verifica quais colunas de log_cols estão presentes no dataset
existing_log_cols = [col for col in log_cols if col in dataset.columns]

# Mostra as colunas que serão transformadas
print(f"{len(existing_log_cols)} Colunas foram encontradas para aplicar log1p:", existing_log_cols)

# Aplica log1p apenas nas colunas presentes
dataset[existing_log_cols] = dataset[existing_log_cols].apply(lambda x: np.log1p(x))

In [ ]:
train = dataset[dataset['split'] == "train"]
test  = dataset[dataset['split'] == "test"]

# Remove as colunas indesejadas
train = train.drop(columns=['split', 'Municipio']) # O nome do municipio serve apenas para ver o resultado do teste para o municipio especifico
test  = test.drop(columns=['split'])

# separa as features numericas e categoricas
alvo = 'target' # nome da coluna alvo
num_features = train.select_dtypes(include=['float64', 'int64']).columns.tolist()
if alvo in num_features:
  num_features.remove(alvo)
cat_features = ['Mesorregião']
print(f'num_features: {num_features}\ncat_features: {cat_features}\nalvo: {alvo}\n\n')

X_train = train[num_features + cat_features]
y_train = train[alvo]
X_test  = test[num_features + cat_features + ['Municipio']]
y_test  = test[alvo]

print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print(f'X_test:  {X_test.shape}')
print(f'y_test:  {y_test.shape}')

# 10. Pipeline de pré-processamento

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ('num', RobustScaler(), num_features),
    ('cat', OneHotEncoder(sparse_output=False, drop='first'), cat_features)
])

# 11. Função para criar o modelo (com stub para Grid Search)

In [ ]:
def make_model(best_params=None, use_grid_search=False):
    rf = RandomForestRegressor(n_estimators=150, random_state=42)

    if not use_grid_search:
        return Pipeline([
            ('prep', preprocessor),
            ('model', rf)
        ])
    else:
        # stub para future grid search
        param_grid = {
          'model__n_estimators': [100, 200],
          'model__max_depth': [None, 10, 20],
          'model__max_features': ['log2', 'sqrt'],
          'model__min_samples_split': [2, 5],
          'model__min_samples_leaf': [1, 2, 4],
          'model__bootstrap': [True, False]
        }

        return GridSearchCV(
            Pipeline([('prep', preprocessor), ('model', rf)]),
            param_grid=param_grid,
            cv=5,
            scoring='neg_root_mean_squared_error',
            n_jobs=-1
        )

# 12. Treinamento

In [ ]:
pipeline = make_model(use_grid_search=False)
pipeline.fit(X_train, y_train)

In [ ]:
import pickle
from datetime import datetime

data_hora_atual = datetime.now()
formato_desejado = data_hora_atual.strftime('%d-%m-%Y-%H-%M')
with open(f'modelo_{formato_desejado}.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

In [ ]:
def get_feature_names(column_transformer):
    feature_names = []

    for name, transformer, cols in column_transformer.transformers_:
        if transformer == 'drop':
            continue
        if transformer == 'passthrough':
            feature_names.extend(cols)
        else:
            # Tenta usar o método get_feature_names_out (sklearn ≥1.0)
            try:
                names = transformer.get_feature_names_out(cols)
            except AttributeError:
                # fallback: usa nomes originais
                names = cols
            feature_names.extend(names)
    return feature_names

# Exemplo de uso:
feature_names = get_feature_names(pipeline.named_steps['prep'])


# 2) Extraia importâncias
importancias = pipeline.named_steps['model'].feature_importances_

# 3) Recupere nomes de features transformadas
feature_names = get_feature_names(pipeline.named_steps['prep'])

# 4) Monte o DataFrame
feat_imp_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importancias
})

# 5) Ordene decrescentemente
feat_imp_df = feat_imp_df.sort_values(by='importance', ascending=False).reset_index(drop=True)

print(feat_imp_df)

# 13. Predição e avaliação

In [ ]:
import pickle

with open('/content/modelo_19_5.pkl', 'rb') as f:
  modelo_carregado = pickle.load(f)

In [ ]:
if True:
  modelo_carregado = pipeline

y_pred_log = modelo_carregado.predict(X_test.drop(columns=['Municipio']))
# inverter log1p: exp(y) - 1
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)


# Cálculo de métricas
r2  = r2_score(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

# Estatísticas da produção real (mil toneladas)
mean_true = np.mean(y_true)
std_true  = np.std(y_true, ddof=0)  # ddof=0 para população

# Índices de interpretação
# 1) RMSE / Desvio-padrão
rmse_std_ratio = rmse / std_true

# 2) RMSE / Média (erro médio relativo)
rmse_mean_ratio = rmse / mean_true

# Impressão dos resultados
print(f'R² no teste:             {r2:.3f}')
print(f'RMSE no teste:           {rmse:.3f} Toneladas/Hectare')
print(f'Média da produção real:  {mean_true:.3f} Toneladas/Hectare')
print(f'Desvio‑padrão real:      {std_true:.3f} Toneladas/Hectare\n')

print('=== Interpretação ===')
print(f'- RMSE ≈ {rmse_std_ratio:.2f} × desvio‑padrão dos dados')
print(f'  (se <1, erros menores que a variação típica)')
print(f'- RMSE ≈ {rmse_mean_ratio:.2%} da média dos dados')
print(f'  (percentual médio de erro em relação à média)')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Criando DataFrame
df = pd.DataFrame({
    'Observado': y_true,
    'Estimado': y_pred
})

# Estatísticas descritivas
estatisticas = df.describe().loc[['mean', 'min', '25%', '50%', '75%', 'max']]
print("Estatísticas Descritivas:\n", estatisticas)

# Métricas de avaliação
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"\nRMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.2f}")

In [ ]:
import matplotlib.pyplot as plt

# 1. Observado vs Estimado
plt.figure(figsize=(6, 6))
plt.scatter(df['Observado'], df['Estimado'], alpha=0.5)
lims = [
    min(df['Observado'].min(), df['Estimado'].min()),
    max(df['Observado'].max(), df['Estimado'].max())
]
plt.plot(lims, lims, linestyle='dashed')
plt.xlabel('Observado')
plt.ylabel('Estimado')
plt.title('Observado vs Estimado - (Toneladas/Hectare)')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets, HBox, VBox, Output
from IPython.display import display

# Widget de saída
out = Output()

# Função de plotagem atualizada
def plot_real_vs_pred(df_test, y_true, y_pred, mesorregiao, municipio):
    mask = (df_test['Mesorregião'] == mesorregiao) & (df_test['Municipio'] == municipio)
    df_f = df_test.loc[mask].copy()
    df_f['Real']     = y_true[mask]
    df_f['Previsto'] = y_pred[mask]

    # Cálculo da diferença percentual
    df_f['Dif_pct'] = np.where(df_f['Real'] != 0,
                               ((df_f['Previsto'] - df_f['Real']) / df_f['Real'] * 100).round(2),
                               np.nan)

    # Cálculo da previsão média dos dois anos anteriores
    df_f.sort_values('Ano', inplace=True)
    df_f['Prev_2yr_mean'] = (
        df_f.set_index('Ano')['Real']
        .rolling(window=3, min_periods=3)
        .apply(lambda x: x.iloc[:-1].mean(), raw=False)
        .values
    )

    # Construir figura com tamanho fixo
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_f['Ano'], y=df_f['Real'],
                             mode='lines+markers', name='Real',
                             line=dict(color='green')))
    fig.add_trace(go.Scatter(x=df_f['Ano'], y=df_f['Previsto'],
                             mode='lines+markers', name='Previsto (Modelo)',
                             line=dict(color='red')))
    fig.add_trace(go.Scatter(x=df_f['Ano'], y=df_f['Dif_pct'],
                             mode='lines+markers', name='% Diferença',
                             line=dict(color='blue', dash='dash')))
    fig.add_trace(go.Scatter(x=df_f.loc[df_f['Prev_2yr_mean'].notna(), 'Ano'],
                             y=df_f.loc[df_f['Prev_2yr_mean'].notna(), 'Prev_2yr_mean'],
                             mode='lines+markers', name='Previsto (Média 2 anos)',
                             line=dict(color='orange', dash='dot')))

    anos_presentes = sorted(df_f['Ano'].unique())
    fig.update_layout(
        title=f'Produção Real vs Prevista — {municipio} ({mesorregiao})',
        xaxis=dict(title='Ano',
                   tickmode='array',
                   tickvals=anos_presentes,
                   ticktext=[str(a) for a in anos_presentes]),
        yaxis_title='Quantidade Produzida / % Diferença',
        width=1500,
        height=600,
        template='plotly_white'
    )
    return fig

# Dropdown de mesorregiões
meso_vals = np.sort(X_test['Mesorregião'].unique())
meso_sel = widgets.Dropdown(
    options=meso_vals,
    description='Mesorregião:',
    style={'description_width': 'initial'}
)

# Dropdown de municípios (vai ser atualizado)
mun_sel = widgets.Dropdown(description='Município:', style={'description_width': 'initial'})
btn    = widgets.Button(description='Atualizar Gráfico', button_style='primary')

# Atualiza municípios quando mesorregião muda
def atualiza_muns(*_):
    sel = meso_sel.value
    mun_list = np.sort(X_test.loc[X_test['Mesorregião'] == sel, 'Municipio'].unique())
    mun_sel.options = mun_list
    if len(mun_list): mun_sel.value = mun_list[0]

meso_sel.observe(atualiza_muns, names='value')
atualiza_muns()

# Callback do botão
def on_btn_click(_):
    with out:
        out.clear_output(wait=True)
        fig = plot_real_vs_pred(X_test, y_true, y_pred,
                                meso_sel.value, mun_sel.value)
        fig.show()

btn.on_click(on_btn_click)

# Exibe interface
ui = VBox([HBox([meso_sel, mun_sel, btn])])
display(ui, out)

# Espinosa - MG (Norte de Minas)

In [ ]:
import pandas as pd
import numpy as np

# ─── Função 1: Calcular média móvel histórica de 2 anos ───────────────────────
def calc_prev_2yr(series_real: pd.Series) -> pd.Series:
    """
    Recebe uma série de valores reais ordenada por ano e retorna
    uma série com a previsão histórica: média dos dois anos anteriores.
    """
    return (
        series_real
        .rolling(window=3, min_periods=3)
        .apply(lambda x: x.iloc[:-1].mean(), raw=False)
    )

# ─── Função 2: Calcular erros percentuais de uma previsão ────────────────────
def calc_pct_error(pred: pd.Series, real: pd.Series) -> pd.Series:
    """
    Retorna o erro percentual entre previsão e valor real:
      erro_pct = ((pred - real) / real) * 100, arredondado em 2 casas.
    """
    pct = np.where(
        real != 0,
        ((pred - real) / real * 100).round(2),
        np.nan
    )
    return pd.Series(pct, index=real.index)

# ─── Função 3: Agregar média de erro percentual por município ────────────────
def mean_pct_error_by_mun(df: pd.DataFrame, col_error: str) -> pd.Series:
    """
    Dado DataFrame com coluna de erro percentual (col_error) e 'Municipio',
    retorna Série com a média desse erro para cada município.
    """
    return df.groupby('Municipio')[col_error].mean()

# ─── Carregamento / Preparação dos dados ────────────────────────────────────
df = X_test[['Municipio', 'Mesorregião', 'Ano']].copy()
df['Real']             = y_true
df['Predição Modelo']  = y_pred

# ─── Etapa 1: Cálculo da previsão histórica ─────────────────────────────────
df = df.sort_values(['Municipio', 'Ano']).reset_index(drop=True)
df['Predição Média 2 anos'] = (
    df
    .groupby('Municipio')['Real']
    .apply(calc_prev_2yr)
    .values
)

# ─── Etapa 2: Cálculo dos erros percentuais ─────────────────────────────────
df['Diferença Modelo (%)']       = calc_pct_error(df['Predição Modelo'], df['Real'])
df['Diferença Média 2 anos (%)'] = calc_pct_error(df['Predição Média 2 anos'], df['Real'])

# ─── Etapa 3: Médias de erro percentual por município ───────────────────────
media_pct_mun_modelo = mean_pct_error_by_mun(df, 'Diferença Modelo (%)') \
                            .rename('Média Percentual Município (%)')
media_pct_mun_media2 = mean_pct_error_by_mun(df, 'Diferença Média 2 anos (%)') \
                            .rename('Média 2 anos Município (%)')

# ─── Etapa 4: Média global de erro percentual do modelo ────────────────────
media_global_modelo = media_pct_mun_modelo.mean()

# ─── Etapa 4.1: Média global de erro percentual da média 2 anos ────────────
media_global_media2 = media_pct_mun_media2.mean()

# ─── Etapa 5: Consolidar métricas no DataFrame final ────────────────────────
df = (
    df
    .merge(media_pct_mun_modelo, on='Municipio', how='left')
    .merge(media_pct_mun_media2,  on='Municipio', how='left')
    .assign(
        **{
            'Média Global Modelo (%)': media_global_modelo,
            'Média Global Média 2 anos (%)': media_global_media2
        }
    )
)

# ─── Etapa 6: Reordenar colunas para exportação ──────────────────────────────
df_final = df[[
    'Municipio',
    'Mesorregião',
    'Ano',
    'Real',
    'Predição Modelo',
    'Diferença Modelo (%)',
    'Predição Média 2 anos',
    'Diferença Média 2 anos (%)',
    'Média Percentual Município (%)',
    'Média 2 anos Município (%)',
    'Média Global Modelo (%)',
    'Média Global Média 2 anos (%)'
]]

# ─── Etapa 6.1: Arredondar valores numéricos ─────────────────────────────────
# 'Real' com 6 casas decimais
#   Predições com 4 casas decimais
#   Erros e médias percentuais com 2 casas decimais
df_final = df_final.assign(
    **{
        'Real': df_final['Real'].round(6),
        'Predição Modelo': df_final['Predição Modelo'].round(4),
        'Predição Média 2 anos': df_final['Predição Média 2 anos'].round(4),
        'Diferença Modelo (%)': df_final['Diferença Modelo (%)'].round(2),
        'Diferença Média 2 anos (%)': df_final['Diferença Média 2 anos (%)'].round(2),
        'Média Percentual Município (%)': df_final['Média Percentual Município (%)'].round(2),
        'Média 2 anos Município (%)': df_final['Média 2 anos Município (%)'].round(2),
        'Média Global Modelo (%)': df_final['Média Global Modelo (%)'].round(2),
        'Média Global Média 2 anos (%)': df_final['Média Global Média 2 anos (%)'].round(2),
    }
)

# ─── Etapa 7: Exportar para .xlsx (recomendado) ─────────────────────────────
df_final.to_excel('resultados_por_municipio.xlsx', index=False, sheet_name='Resumo')
print("Arquivo 'resultados_por_municipio.xlsx' salvo com sucesso!")

In [ ]:
df_final.head(15)

#14. Se for usar Grid Search


In [ ]:
grid_pipeline = make_model(use_grid_search=True)
grid_pipeline.fit(X_train, y_train)
print("Melhores parâmetros:", grid_pipeline.best_params_)

In [ ]:
grid_pipeline = make_model(use_grid_search=False)
grid_pipeline.fit(X_train, y_train)

In [ ]:
y_pred_log = pipeline.predict(X_test)
# inverter log1p: exp(y) - 1
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)


# Cálculo de métricas
r2  = r2_score(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

# Estatísticas da produção real (mil toneladas)
mean_true = np.mean(y_true)
std_true  = np.std(y_true, ddof=0)  # ddof=0 para população

# Índices de interpretação
# 1) RMSE / Desvio-padrão
rmse_std_ratio = rmse / std_true

# 2) RMSE / Média (erro médio relativo)
rmse_mean_ratio = rmse / mean_true

# Impressão dos resultados
print(f'R² no teste:             {r2:.3f}')
print(f'RMSE no teste:           {rmse:.3f} mil toneladas')
print(f'Média da produção real:  {mean_true:.3f} mil toneladas')
print(f'Desvio‑padrão real:      {std_true:.3f} mil toneladas\n')

print('=== Interpretação ===')
print(f'- RMSE ≈ {rmse_std_ratio:.2f} × desvio‑padrão dos dados')
print(f'  (se <1, erros menores que a variação típica)')
print(f'- RMSE ≈ {rmse_mean_ratio:.2%} da média dos dados')
print(f'  (percentual médio de erro em relação à média)')
print(f'\n\nResultado do artigo: R² de 0.822 e RMSE (mil ton) 0.177')

# outros

In [ ]:
import os
import shutil
import numpy as np
import plotly.graph_objects as go

def plot_real_vs_pred_fig(df_f, mesorregiao, municipio):
    df_f = df_f.sort_values('Ano').copy()
    # Diferença percentual
    df_f['Dif_pct'] = np.where(
        df_f['Real'] != 0,
        ((df_f['Previsto'] - df_f['Real']) / df_f['Real'] * 100).round(2),
        np.nan
    )
    # Média móvel dos 2 anos anteriores
    df_f['Prev_2yr_mean'] = (
        df_f.set_index('Ano')['Real']
          .rolling(window=3, min_periods=3)
          .apply(lambda x: x.iloc[:-1].mean(), raw=False)
          .values
    )

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_f['Ano'], y=df_f['Real'],
                             mode='lines+markers', name='Real',
                             line=dict(color='green')))
    fig.add_trace(go.Scatter(x=df_f['Ano'], y=df_f['Previsto'],
                             mode='lines+markers', name='Previsto (Modelo)',
                             line=dict(color='red')))
    fig.add_trace(go.Scatter(x=df_f['Ano'], y=df_f['Dif_pct'],
                             mode='lines+markers', name='% Diferença',
                             line=dict(color='blue', dash='dash')))
    df_mm = df_f[df_f['Prev_2yr_mean'].notna()]
    fig.add_trace(go.Scatter(x=df_mm['Ano'], y=df_mm['Prev_2yr_mean'],
                             mode='lines+markers', name='Previsto (Média 2 anos)',
                             line=dict(color='orange', dash='dot')))

    anos = sorted(df_f['Ano'].unique())
    fig.update_layout(
        title=f'Produção Real vs Prevista — {municipio} ({mesorregiao})',
        xaxis=dict(title='Ano',
                   tickmode='array',
                   tickvals=anos,
                   ticktext=[str(a) for a in anos]),
        yaxis_title='Quantidade Produzida / % Diferença',
        width=1500, height=600,
        template='plotly_white'
    )
    return fig

def generate_and_zip_graphs(X_test, y_true, y_pred,
                            base_path='graficos', zip_name='graficos_zip'):
    os.makedirs(base_path, exist_ok=True)
    # Agora iteramos sobre os valores únicos de “Mesorregião”
    meso_vals = np.sort(X_test['Mesorregião'].unique())

    for meso in meso_vals:
        meso_dir = os.path.join(base_path, meso)
        os.makedirs(meso_dir, exist_ok=True)

        mask_meso = X_test['Mesorregião'] == meso
        municipios = np.sort(X_test.loc[mask_meso, 'Municipio'].unique())

        for mun in municipios:
            mask = mask_meso & (X_test['Municipio'] == mun)
            df_f = X_test.loc[mask, ['Ano']].copy()
            df_f['Real']     = y_true[mask]
            df_f['Previsto'] = y_pred[mask]

            fig = plot_real_vs_pred_fig(df_f, meso, mun)

            filename = f"{mun.replace('/', '_').replace(' ', '_')}.html"
            filepath = os.path.join(meso_dir, filename)
            fig.write_html(filepath, include_plotlyjs='cdn')

    zip_path = shutil.make_archive(zip_name, 'zip', base_path)
    print(f"\n👉 Arquivo ZIP criado em: {os.path.abspath(zip_path)}")


generate_and_zip_graphs(X_test, y_true, y_pred,
                         base_path='graficos_municipios',
                         zip_name='graficos_municipios')